In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import route_dynamics.route_elevation.base as base
import geopandas as gpd

from shapely.geometry import mapping

In [3]:
dat1 = pd.read_csv("../data/Trip183.csv")
dat2 = pd.read_csv("../data/Zon183Unsum.csv")

/Users/ryanjcarlin/miniconda3/envs/simple_route_dynamics/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
trip183 = dat1[['SignRt', 'InOut', 'KeyTrip', 'BusType', 'Seats',
                'Period', 'AnnRides']]

In [5]:
trip183unsum = dat2[['Route', 'Dir', 'Trip_ID', 'InOut', 'STOP_SEQ', 'STOP_ID',
                     'Period', 'AveOn', 'AveOff', 'AveLd', 'Obs']]

In [6]:
bus_mass = {
    11: 11000,
    26: 19051,
    32: 11793,
    36: 11793,
    37: 12247,
    43: 14913,
    45: 19051,
    46: 18835,
    60: 19051,
    62: 19051,
    68: 19051,
    70: 12927,
    72: 12927,
    73: 12927,
    80: 19051,
    81: 19051,
    82: 19051,
    90: 12927,
    91: 12927,
    92: 12927,
    95: 19051,
    96: 19051
        }

In [7]:
trip183 = trip183.replace({'BusType': bus_mass})
trip_mass = trip183[['BusType', 'KeyTrip']]
trip_dict = dict(zip(trip_mass.KeyTrip, trip_mass.BusType))

In [8]:
def route_ridership(period, direction, route):
    
    df = trip183unsum
    df = df.drop(df[(df.Period != period)].index)
    df = df.drop(df[(df.InOut != direction)].index)
    df = df.drop(df[(df.Route != route)].index)

    final_df = df.sort_values(by=['Trip_ID', 'STOP_SEQ', 'STOP_ID'])
    seq_id = final_df[['STOP_SEQ', 'STOP_ID']]
    seq_id2 = seq_id.drop_duplicates(subset=['STOP_SEQ'], keep='first')
    seq_id3 = seq_id2.sort_values(by='STOP_SEQ')
    stopid_dic = dict(zip(seq_id3.STOP_SEQ, seq_id3.STOP_ID))
    riders = final_df.pivot(index='STOP_SEQ', columns='Trip_ID', values='AveLd')

    keyfind = list(riders.columns)
    mass_bus = [trip_dict[x] for x in keyfind]
    mode = mode_mass = max(set(mass_bus), key=mass_bus.count)
    rider_columns = list(riders.columns)
    kept_columns = []

    for i in range(0, len(mass_bus)):
        if mass_bus[i] == mode:
            kept_columns.append(rider_columns[i])

    riders_interm = riders[kept_columns]
    riders_kept = pd.DataFrame((riders_interm.mean(axis=1)), columns=['Mean'])
    riders_kept.Mean*=80
    riders_kept.reset_index(inplace=True)
    riders_kept.replace({"STOP_SEQ": stopid_dic}, inplace=True)
    riders_kept.columns = ['STOP_ID', 'Mean']



    return final_df, riders_kept, mode_mass

In [9]:
df_45, riders_45, mass_45 = route_ridership('AM', 'I', 45)

In [10]:
riders_45

,STOP_ID,Mean
0,35530.0,43.692308
1,35540.0,74.461538
2,35550.0,94.153846
3,19210.0,122.461538
4,35551.0,158.769231
5,35570.0,193.846154
6,35580.0,227.076923
7,35600.0,353.846154
8,35630.0,404.307692
9,41430.0,516.307692


In [11]:
routes_shp = '../data/six_routes.shp'
stops_shp = '../data/Transit_Stops_for_King_County_Metro__transitstop_point.shp'

In [40]:
def stop_coord(num, riders_num):
    route_num = num
    
    route = base.read_shape(routes_shp, route_num)
    points = base.extract_point_df(route)
    stops = gpd.read_file(stops_shp)
    stops['ROUTE_LIST'].fillna(value=str(0), inplace=True)
    
    stops_list = pd.DataFrame()
    for i in range(0, len(stops)):
    
        if str(route_num) in (stops['ROUTE_LIST'][i]):
            for x in stops['ROUTE_LIST'][i].split(' '):
                if str(route_num) == x:
                    stops_list = stops_list.append(stops.iloc[i])
                else:
                    pass
        else:
            pass
        
    stop_ids = stops_list['STOP_ID'].values
    
    geometry = stops_list.geometry.values

    xy = []
    for i in range(len(geometry)):
        dic = mapping(geometry[i])
        coords = dic['coordinates']
        xy.append(coords)
        xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
        xy_df['coordinates'] = xy
    
    xy_df['STOP_ID'] = stops_list['STOP_ID'].values
    
    df = riders_num
    df_ind = df.reset_index()
    df_ind.columns = ['STOP_SEQ', 'STOP_ID', 'Mean']
    
    df_comb = xy_df.merge(df_ind, how='right', on="STOP_ID")
    df_comb = df_comb.drop_duplicates(subset=['STOP_ID'], keep='first')
    df_comb = df_comb[['STOP_SEQ', 'STOP_ID', 'coordinates', 'Mean']]
    df_combine = df_comb.sort_values(by='STOP_SEQ')
    
    return xy_df, df_combine

In [44]:
coordinates_45, final_45 = stop_coord(45, riders_45)

In [45]:
coordinates_45.head()

,STOP_ID,coordinates
0,5330.0,"(-122.35493491168019, 47.69052891200809)"
1,5350.0,"(-122.34960617887032, 47.6905094486964)"
2,5370.0,"(-122.34472418112374, 47.69049011915158)"
3,5380.0,"(-122.34150318558568, 47.69047109565094)"
4,5400.0,"(-122.33660476604088, 47.6904450549253)"


In [46]:
final_45.head()

,STOP_SEQ,STOP_ID,coordinates,Mean
56,0,35530.0,"(-122.39786618471946, 47.690295853436986)",43.692308
58,1,35540.0,"(-122.39653218098778, 47.690500893105884)",74.461538
60,2,35550.0,"(-122.3941645942555, 47.69050290761158)",94.153846
36,3,19210.0,"(-122.39161830818995, 47.69050457451431)",122.461538
62,4,35551.0,"(-122.38776029322838, 47.690506809007324)",158.769231
